In [1]:
"""
Boilerplate to implement training different networks on different datasets
with varying config.

I wish a machine could automate setting up decent baseline models and datasets
"""
#  import json
import os
from os.path import exists
import pampy
from simple_parsing import ArgumentParser, choice
from simplepytorch import datasets as D
from simplepytorch import trainlib as TL
from simplepytorch import metrics
from sklearn.model_selection import StratifiedShuffleSplit
from torch.utils.data import Dataset, DataLoader
from typing import Union, Optional
import dataclasses as dc
import numpy as np
import torch as T
import torchvision.transforms as tvt
import torch
from torch import optim, nn
from torchvision import models, transforms

from deepfix.models import get_effnetv2, get_resnet, get_efficientnetv1, get_DeepFixEnd2End, DeepFixMLP
from deepfix.models.ghaarconv import convert_conv2d_to_gHaarConv2d
from deepfix.init_from_distribution import init_from_beta, reset_optimizer
from deepfix import deepfix_strategies as dfs
import pytorch_wavelets as pyw

In [2]:
import numpy as np
import scipy.io
# from ipynb.fs.defs.q2 import *
import sklearn
import torch
import torchvision.datasets
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import pandas as pd
# YOUR CODE HERE
# Use GPU if available, otherwise stick with cpu
use_cuda = torch.cuda.is_available()
torch.manual_seed(123)
device = torch.device("cuda" if use_cuda else "cpu")
print("device = {}".format(device))

device = cuda


In [3]:
# !ln -sr /ocean/projects/cie160013p/agaudio/data/CheXpert-v1.0-small ~/store/DeepFix/data/

In [4]:
# import sys
# sys.executable

In [8]:
MODELS = {
    ('effnetv2', str, str, str): (
        lambda pretrain, in_ch, out_ch: get_effnetv2(pretrain, int(in_ch), int(out_ch))),
    ('resnet50', str, str, str): (
        lambda pretrain, in_ch, out_ch: get_resnet('resnet50', pretrain, int(in_ch), int(out_ch))),
    ('resnet18', str, str, str): (
        lambda pretrain, in_ch, out_ch: get_resnet('resnet18', pretrain, int(in_ch), int(out_ch))),
    ('efficientnet-b0', str, str, str): (
        lambda pretrain, in_ch, out_ch: get_efficientnetv1('efficientnet-b0', pretrain, int(in_ch), int(out_ch))),
    ('efficientnet-b1', str, str, str): (
        lambda pretrain, in_ch, out_ch: get_efficientnetv1('efficientnet-b1', pretrain, int(in_ch), int(out_ch))),
    ('waveletres18', str, str, str): lambda pretrain, in_ch, out_ch: R(
        pretrain, int(in_ch), int(out_ch)),
    ('waveletmlp', str, str, str, str, str, str, str): (
        lambda mlp_channels, in_ch, out_ch, wavelet_levels, patch_size, in_ch_mul, mlp_depth: get_DeepFixEnd2End(
            int(in_ch), int(out_ch),
            in_ch_multiplier=int(in_ch_mul), wavelet='db1',
            wavelet_levels=int(wavelet_levels), wavelet_patch_size=int(patch_size),
            mlp_depth=int(mlp_depth), mlp_channels=int(mlp_channels),
            mlp_fix_weights='none', mlp_activation=None)
        ),

    #  ('waveletres18v2', str, str, str): lambda pretrain, in_ch, out_ch: (
        #  DeepFixCompression(levels=8, wavelet='coif1', patch_size=1),
        #  R2(pretrain, int(in_ch), int(out_ch))),
}


class R(T.nn.Module):
    def __init__(self, pretrain, in_ch, out_ch):
        super().__init__()
        self.r = get_resnet('resnet18', pretrain, in_ch, out_ch,)
        self.dwt = pyw.DWT(J=8, wave='coif1', mode='zero')

    @staticmethod
    def wavelet_coefficients_as_tensorimage(approx, detail, normalize=False):
        B,C = approx.shape[:2]
        fixed_dims = approx.shape[:-2] # num images in minibatch, num channels, etc
        output_shape = fixed_dims + (
            detail[0].shape[-2]*2,  # input img height
            detail[0].shape[-1]*2)  # input img width
        im = T.zeros(output_shape, device=approx.device, dtype=approx.dtype)
        if normalize:
            norm11 = lambda x: (x / max(x.min()*-1, x.max()))  # into [-1,+1] preserving sign
            #  approx = norm11(approx)
        im[..., :approx.shape[-2], :approx.shape[-1]] = approx if approx is not None else 0
        for level in detail:
            lh, hl, hh = level.unbind(-3)
            h,w = lh.shape[-2:]
            if normalize:
                lh, hl, hh = [norm11(x) for x in [lh, hl, hh]]
            #  im[:h, :w] = approx
            im[..., 0:h, w:w+w] = lh  # horizontal
            im[..., h:h+h, :w] = hl  # vertical
            im[..., h:h+h, w:w+w] = hh  # diagonal
        return im

    def forward(self, x):
        x = self.wavelet_coefficients_as_tensorimage(*self.dwt(x))
        return self.r(x)


class R2(T.nn.Module):
    def __init__(self, pretrain, in_ch, out_ch):
        super().__init__()
        self.r = get_resnet('resnet18', pretrain, in_ch, out_ch,)

    def forward(self, x):
        B,C,H = x.shape
        x = x.unsqueeze(-1).repeat(1,1,1,H)
        return self.r(x)


class LossCheXpertIdentity(T.nn.Module):
    def __init__(self, N):
        super().__init__()
        self.bce = T.nn.BCEWithLogitsLoss()
        self.N = N

    def forward(self, yhat, y):
        # absolute max possible num patients in chexpert is 223414
        # but let's just hash them into a smaller number of bins via modulo N
        assert self.N == yhat.shape[1], \
                f'note: model must have {self.N} binary predictions per sample'
        y_onehot = y.new_zeros(y.shape[0], self.N, dtype=T.float
                               ).scatter_(1, y.long()%self.N, 1)
        return self.bce(yhat[:, -1], y_onehot[:, -1])


class LossCheXpertUignore(T.nn.Module):
    def __init__(self):
        super().__init__()
        self.bce = T.nn.BCEWithLogitsLoss()

    def forward(self, yhat, y):
        ignore = (y != 2)  # ignore uncertainty labels
        return self.bce(yhat[ignore], y[ignore])


def loss_intelmobileodt(yhat, y):
    """BCE Loss with class balancing weights.

    Not sure this actually helps

    because Type 2 is the hardest class, it
    has the most samples, and it separates Type 1 from Type 3.  Arguably, Type 2
    samples are on the decision boundary between Type 1 and 3.
    Class balancing weights make it harder to focus on class 2.
    """
    #  assert y.shape == yhat.shape, 'sanity check'
    #  assert y.dtype == yhat.dtype, 'sanity check'

    # class distribution of stage='train'
    w = T.tensor([249, 781, 450], dtype=y.dtype, device=y.device)
    w = (w.max() / w).reshape(1, 3)
    # w can have any of the shapes:  (B,1) or (1,C) or (B,C)
    #  return T.nn.functional.binary_cross_entropy_with_logits(yhat, y, weight=w)
    return T.nn.functional.cross_entropy(yhat, y, weight=w)
    # can't apply focal loss unless do it manually.


def onehot(y, nclasses):
    return T.zeros((y.numel(), nclasses), dtype=y.dtype, device=y.device)\
            .scatter_(1, y.unsqueeze(1), 1)


def _upsample_pad_minibatch_imgs_to_same_size(batch, target_is_segmentation_mask=False):
    """a collate function for a dataloader of (x,y) samples.  """
    shapes = [item[0].shape for item in batch]
    H = max(h for c,h,w in shapes)
    W = max(w for c,h,w in shapes)
    X, Y = [], []
    for item in batch:
        h,w = item[0].shape[1:]
        dh, dw = (H-h), (W-w)
        padding = (dw//2, dw-dw//2, dh//2, dh-dh//2, )
        X.append(T.nn.functional.pad(item[0], padding))
        if target_is_segmentation_mask:
            Y.append(T.nn.functional.pad(item[1], padding))
        else:
            Y.append(item[1])
    return T.stack(X), T.stack(Y)


def get_dset_chexpert(train_frac=.8, val_frac=.2, small=False,
                      labels:str='diagnostic', num_identities=None):
    """
    Args:
        labels:  either "diagnostic" (the 14 classes defined as
            D.CheXpert.LABELS_DIAGNOSTIC) or "identity" ("patient", "study",
            "view", "index")
        small:  whether to use CheXpert_Small dataset (previously downsampled
            images) or the fully size dataset.
        num_identities:  used only if labels='identity'.  If
            num_identities=1000, then all patients get identified as coming
            from precisely 1 of 1000 bins.

    Returns:
        (
        {'train_dset': ..., 'val_dset': ..., 'test_dset': ...,
         'train_loader': ..., 'val_loader': ..., 'test_loader': ...
         },

        ('Pneumonia', 'Cardiomegaly', ...)  # class names defined by `labels`
        )
    """
    _label_cleanup_dct = dict(D.CheXpert.LABEL_CLEANUP_DICT)
    if labels == 'diagnostic':
        class_names = D.CheXpert.LABELS_DIAGNOSTIC
        for k in class_names:
            _label_cleanup_dct[k][np.nan] = 0  # remap missing value to negative
        get_ylabels = lambda dct: \
                D.CheXpert.format_labels(dct, labels=class_names).float()
    elif labels == 'identity':
        class_names = list(range(num_identities))
        get_ylabels = lambda dct: \
                (D.CheXpert.format_labels(dct, labels=['index']) % num_identities).long()
    else:
        raise NotImplementedError(f"unrecognized labels: {labels}")
    kws = dict(
        img_transform=tvt.Compose([
            #  tvt.RandomCrop((512, 512)),
            tvt.ToTensor(),  # full res 1024x1024 imgs
            tvt.Resize((224, 224)),
            tvt.Lambda(lambda x: x.repeat(3, 1, 1)),
            
        ]),
        getitem_transform=lambda dct: (dct['image'], get_ylabels(dct)),
        label_cleanup_dct=_label_cleanup_dct,
    )
    if small:
        kls = D.CheXpert_Small
    else:
        kls = D.CheXpert

    train_dset = kls(dataset_dir="../data/CheXpert-v1.0-small/",use_train_set=True, **kws)  #Edited Elvin

    N = len(train_dset)
    if train_frac + val_frac == 1:
        nsplits = [N - int(N*val_frac), int(N*val_frac), 0]
    else:
        a,b = int(N*train_frac), int(N*val_frac)
        nsplits = [a,b, N-a-b]
    train_dset, val_dset, _ = T.utils.data.random_split(train_dset, nsplits)
    test_dset = kls(dataset_dir="../data/CheXpert-v1.0-small/",use_train_set=False, **kws) #Edited Elvin
    batch_dct = dict(
        batch_size=1, collate_fn=_upsample_pad_minibatch_imgs_to_same_size,
        num_workers=int(os.environ.get("num_workers", 4)))  # upsample pad must take time
    train_loader=DataLoader(train_dset, shuffle=True, **batch_dct)
    val_loader=DataLoader(val_dset, **batch_dct)
    test_loader=DataLoader(test_dset, **batch_dct)
    return (dict(
        train_dset=train_dset, val_dset=val_dset, test_dset=test_dset,
        train_loader=train_loader, val_loader=val_loader, test_loader=test_loader,
    ), class_names)



def match(spec:str, dct:dict):
    return pampy.match(spec.split(':'), *(x for y in dct.items() for x in y))


def get_model_opt_loss(
        model_spec:str, opt_spec:str, loss_spec:str, regularizer_spec:str,
        device:str) -> dict[str, Union[T.nn.Module, T.optim.Optimizer]]:
    """
    Args:
        model_spec: a string of form,
            "model_name:pretraining:in_channels:out_classes".  For example:
            "effnetv2:untrained:1:5"
        opt_spec: Specifies how to create optimizer.
            First value is a pytorch Optimizer in T.optim.*.
            Other values are numerical parameters.
            Example: "SGD:lr=.003:momentum=.9"
        device: e.g. 'cpu' or 'gpu'
    Returns:
        a pytorch model and optimizer
    """
    mdl = match(model_spec, MODELS)
    mdl = mdl.to(device, non_blocking=True)
    optimizer = reset_optimizer(opt_spec, mdl)
    loss_fn = match(loss_spec, LOSS_FNS)
    if regularizer_spec != 'none':
        loss_fn = RegularizedLoss(mdl, loss_fn, regularizer_spec)
    return dict(model=mdl, optimizer=optimizer, loss_fn=loss_fn)


class RegularizedLoss(T.nn.Module):
    def __init__(self, model, lossfn, regularizer_spec:str):
        super().__init__()
        self.lossfn = lossfn
        self.regularizer_spec = regularizer_spec
        if regularizer_spec == 'none':
            self.regularizer = lambda *y: 0
        elif regularizer_spec.startswith('deepfixmlp:'):
            lbda = float(regularizer_spec.split(':')[1])
            self.regularizer = lambda *y: (
                float(lbda) * DeepFixMLP.get_VecAttn_regularizer(model))
        else:
            raise NotImplementedError(regularizer_spec)

    def forward(self, yhat, y):
        return self.lossfn(yhat, y) + self.regularizer(yhat, y)

    def __repr__(self):
        return f'RegularizedLoss<{repr(self.lossfn)},{self.regularizer_spec}>'


def get_dset_loaders_resultfactory(dset_spec:str) -> dict:
    dct, class_names = match(dset_spec, DSETS)
    if any(dset_spec.startswith(x) for x in {'intel_mobileodt:',
                                             'chexpert_small_ID:'}):
        #  dct['result_factory'] = lambda: TL.MultiLabelBinaryClassification(
                #  class_names, binarize_fn=lambda yh: (T.sigmoid(yh)>.5).long())
        dct['result_factory'] = lambda: TL.MultiClassClassification(
                len(class_names), binarize_fn=lambda yh: yh.softmax(1).argmax(1))
    elif any(dset_spec.startswith(x) for x in {'chexpert:', 'chexpert_small:'}):
        dct['result_factory'] = lambda: CheXpertMultiLabelBinaryClassification(
            class_names, binarize_fn=lambda yh: (yh.sigmoid()>.5).long(), report_avg=True)
    else:
        raise NotImplementedError(f"I don't know how to create the result factory for {dset_spec}")
    return dct

class CheXpertMultiLabelBinaryClassification(TL.MultiLabelBinaryClassification):
    def update(self, yhat, y, loss) -> None:
        self.num_samples += yhat.shape[0]
        self.loss += loss.item()
        assert yhat.shape == y.shape
        assert yhat.ndim == 2 and yhat.shape[1] == len(self._cms), "sanity check: model outputs expected prediction shape"
        binarized = self._binarize_fn(yhat)
        assert binarized.dtype == T.long, 'sanity check binarize fn'
        assert binarized.shape == y.shape, 'sanity check binarize fn'
        ignore = (y != 2)  # ignore uncertainty labels
        for i, (kls, cm) in enumerate(self._cms.items()):
            rows = ignore[:, i]
            if rows.sum() == 0:
                continue  # don't update a confusion matrix if all data for this class is ignored
            cm += metrics.confusion_matrix(y[rows, i], binarized[rows, i], num_classes=2).cpu()


def get_deepfix_train_strategy(args:'TrainOptions'):
    deepfix_spec = args.deepfix
    if deepfix_spec == 'off':
        return TL.train_one_epoch
    elif deepfix_spec.startswith('reinit:'):
        _, N, P, R = deepfix_spec.split(':')
        return dfs.DeepFix_TrainOneEpoch(int(N), float(P), int(R), TL.train_one_epoch)
    elif deepfix_spec.startswith('dhist:'):
        fp = deepfix_spec.split(':', 1)[1]
        assert exists(fp), f'histogram file not found: {fp}'
        return dfs.DeepFix_DHist(fp)
    elif deepfix_spec.startswith('dfhist:'):
        fp = deepfix_spec.split(':', 1)[1]
        assert exists(fp), f'histogram file not found: {fp}'
        return dfs.DeepFix_DHist(fp, fixed=True)
    elif deepfix_spec == 'fixed':
        return dfs.DeepFix_DHist('', fixed=True, init_with_hist=False)
    elif deepfix_spec.startswith('beta:'):
        alpha, beta = deepfix_spec.split(':')[1:]
        return dfs.DeepFix_LambdaInit(
            lambda cfg: init_from_beta(cfg.model, float(alpha), float(beta)))
    elif deepfix_spec.startswith('ghaarconv2d:'):
        ignore_layers = deepfix_spec.split(':')[1].split(',')
        return dfs.DeepFix_LambdaInit(
            lambda cfg: (
                print(f'initialize {deepfix_spec}'),
                convert_conv2d_to_gHaarConv2d(cfg.model, ignore_layers=ignore_layers),
                reset_optimizer(args.opt, cfg.model),
                print(cfg.model)
            ))
    else:
        raise NotImplementedError(deepfix_spec)


def train_config(args:'TrainOptions') -> TL.TrainConfig:
    return TL.TrainConfig(
        **get_model_opt_loss(
            args.model, args.opt, args.lossfn, args.loss_reg, args.device),
        **get_dset_loaders_resultfactory(args.dset),
        device=args.device,
        epochs=args.epochs,
        start_epoch=args.start_epoch,
        train_one_epoch=get_deepfix_train_strategy(args),
        experiment_id=args.experiment_id,
    )


@dc.dataclass
class TrainOptions:
    """High-level configuration for training PyTorch models
    on the IntelMobileODTCervical dataset.
    """
    epochs:int = 50
    start_epoch:int = 0  # if "--start_epoch 1", then don't evaluate perf before training.
    device:str = 'cuda' if T.cuda.is_available() else 'cpu'
    dset:str = None #choice(
        #  'intel_mobileodt:train:val:test:v1',
        #  'intel_mobileodt:train+additional:val:test:v1',
        #  'intel_mobileodt:train+additional:noval:test:v1',
        #  'chexpert:.8:.2', 'chexpert:.01:.01', 'chexpert:.001:.001',
        #  'chexpert_small:.8:.2', 'chexpert_small:.01:.01',
        #   'chexpert_small:.001:.001',
        #  default='intel_mobileodt:train:val:test:v1')
    opt:str = 'SGD:lr=.001:momentum=.9:nesterov=1'
    lossfn:str = None  # choices:
        #  'BCEWithLogitsLoss',
        #  'CrossEntropyLoss', 
        #  'CE_intelmobileodt',
        #  'chexpert_uignore', 
        #  'chexpert_identity:N' for some N=num_identities predicted by model (compared to identities y%N)
    loss_reg:str = 'none'  # Optionally add a regularizer to the loss.  loss + reg.  Accepted values:  'none', 'deepfixmlp:X' where X is a positive float denoting the lambda in l1 regularizer
    model:str = 'resnet18:imagenet:3:3'  # Model specification adheres to the template "model_name:pretraining:in_ch:out_ch"
    deepfix:str = 'off'  # DeepFix Re-initialization Method.
                         #  "off" or "reinit:N:P:R" or "d[f]hist:path_to_histogram.pth"
                         #  or "beta:A:B" for A,B as (float) parameters of the beta distribution
                         # 'ghaarconv2d:layer1,layer2' Replaces all spatial convolutions with GHaarConv2d layer except the specified layers
    experiment_id:str = os.environ.get('run_id', 'debugging')
    prune:str = 'off'

    def execute(self):
        cfg = train_config(self)
        cfg.train(cfg)


def main():
    p = ArgumentParser()
    p.add_arguments(TrainOptions, dest='TrainOptions')
#     for patch_size in [1,32]:
#         for wavelet_level in [1,2,3,4,5,6,7,8,9]:    
#             try:
    in_ch, out_ch = 3, 3
    model_params = "resnet18:imagenet:"+str(in_ch)+":"+str(in_ch)    
    
#     model_params = "waveletmlp:300:1:14:"+str(patch_size)+":"+str(wavelet_level)+":1:2"
    exp_id = 'model_'+model_params+'_in_ch_'+str(in_ch)+'out_ch_'+str(in_ch)#+'_patch_size_' + str(patch_size) + '_level_' + str(wavelet_level)
    args = p.parse_args(["--dset", "chexpert_small:.01:.01", "--opt", "Adam:lr=0.001", "--lossfn", "chexpert_uignore", "--model", model_params, "--loss_reg", "none","--experiment_id",exp_id]).TrainOptions

    print(args)
    cfg = train_config(args)

# python deepfix/train.py --dset chexpert_small:.01:.01 --opt Adam:lr=0.001 --lossfn chexpert_uignore --model waveletmlp:300:1:14:7:1:1:2 --loss_reg none    

    if args.prune != 'off':
        assert args.prune.startswith('ChannelPrune:')
        raise NotImplementedError('code is a bit hardcoded, so it is not available without hacking on it.')
        print(args.prune)
        from explainfix import channelprune
        from deepfix.weight_saliency import costfn_multiclass
        a = sum([x.numel() for x in cfg.model.parameters()])
        channelprune(cfg.model, pct=5, grad_cost_fn=costfn_multiclass,
                     loader=cfg.train_loader, device=cfg.device, num_minibatches=10)
        b = sum([x.numel() for x in cfg.model.parameters()])
        assert a/b != 1
        print(f'done channelpruning.  {a/b}')

    cfg.train(cfg)
#             except Exception as e:
#                 print("=================================================================================================")
#                 print(e)
#                 print("=================================================================================================")
            
    print('+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')
    #  import IPython ; IPython.embed() ; import sys ; sys.exit()

    #  with T.profiler.profile(
    #      activities=[
    #          T.profiler.ProfilerActivity.CPU,
    #          T.profiler.ProfilerActivity.CUDA,
    #      ], with_modules=True,
    #  ) as p:
    #      cfg.train(cfg)
    #  print(p.key_averages().table(
    #      sort_by="self_cuda_time_total", row_limit=-1))


# if __name__ == "__main__":
#     main()


In [9]:
d,l = get_dset_chexpert()
d

{'train_dset': <torch.utils.data.dataset.Subset at 0x1468202f4c70>,
 'val_dset': <torch.utils.data.dataset.Subset at 0x1468202f4e50>,
 'test_dset': <simplepytorch.datasets.chexpert.CheXpert at 0x146826e4dbe0>,
 'train_loader': <torch.utils.data.dataloader.DataLoader at 0x14682771c850>,
 'val_loader': <torch.utils.data.dataloader.DataLoader at 0x1468202920d0>,
 'test_loader': <torch.utils.data.dataloader.DataLoader at 0x146820292cd0>}

In [10]:
# trainset_loader = d['train_dset']
# valset_loader = d['val_dset']

trainset_loader = d['train_loader']
valset_loader = d['val_loader']

In [11]:
len(trainset_loader)

178732

In [29]:
model = models.resnet18(pretrained=True)
# model

In [30]:
new_model = torch.nn.Sequential(*list(model.children())[:-1])
device = torch.device('cuda:0' if torch.cuda.is_available() else "cpu")
new_model = new_model.to(device)

In [32]:
from tqdm import tqdm
import numpy as np

# Will contain the feature
X_train = []
Y_train = []
for data in tqdm(trainset_loader):
    inputs, labels = data
    inputs = inputs.to(device)
    labels = torch.max(labels, 1)[1].to(device)
    with torch.no_grad():
        # Extract the feature from the image
        feature = new_model(inputs)
        X_train.append(feature.cpu().detach().numpy().reshape(512))
        Y_train.append(labels.cpu().detach().numpy()[0])
X_train = np.array(X_train)
Y_train = np.array(Y_train)

100%|██████████| 178732/178732 [29:07<00:00, 102.26it/s]


In [33]:
# Will contain the feature
X_test = []
Y_test = []
for data in tqdm(valset_loader):
    inputs, labels = data
    inputs = inputs.to(device)
    labels = torch.max(labels, 1)[1].to(device)
    with torch.no_grad():
        # Extract the feature from the image
        feature = new_model(inputs)
#         print(feature.size())
        # Convert to NumPy Array, Reshape it, and save it to features variable
        X_test.append(feature.cpu().detach().numpy().reshape(512))
        Y_test.append(labels.cpu().detach().numpy()[0])
#         features.append(feature.cpu().detach().numpy().reshape(-1,4096))
# Convert to NumPy Array
X_test = np.array(X_test)
Y_test = np.array(Y_test)

100%|██████████| 44682/44682 [07:28<00:00, 99.65it/s] 


In [35]:
pd.DataFrame(X_train).to_csv("feature_csv/resnet18_x_train.csv", index=False)  
pd.DataFrame(Y_train).to_csv("feature_csv/resnet18_y_train.csv", index=False)
# X_train = pd.read_csv("feature_csv/resnet18_x_train.csv")
# Y_train = pd.read_csv("feature_csv/resnet18_y_train.csv")

pd.DataFrame(X_test).to_csv("feature_csv/resnet18_x_test.csv", index=False)  
pd.DataFrame(Y_test).to_csv("feature_csv/resnet18_y_test.csv", index=False)
# X_test = pd.read_csv("feature_csv/resnet18_x_test.csv")
# Y_test = pd.read_csv("feature_csv/resnet18_y_test.csv")

In [109]:
# pd.DataFrame(X_train).to_csv("feature_csv/vgg16_x_train.csv", index=False)  
# pd.DataFrame(Y_train).to_csv("feature_csv/vgg16_y_train.csv", index=False)
X_train = pd.read_csv("feature_csv/resnet18_x_train.csv")
Y_train = pd.read_csv("feature_csv/resnet18_y_train.csv")

# pd.DataFrame(X_test).to_csv("feature_csv/vgg16_x_test.csv", index=False)  
# pd.DataFrame(Y_test).to_csv("feature_csv/vgg16_y_test.csv", index=False)
X_test = pd.read_csv("feature_csv/resnet18_x_test.csv")
Y_test = pd.read_csv("feature_csv/resnet18_y_test.csv")

In [58]:
X_train = X_train.iloc[:,:].values
Y_train = Y_train.iloc[:,:].values

X_test = X_test.iloc[:,:].values
Y_test = Y_test.iloc[:,:].values

In [31]:
ty = Y_train[Y_train !=0]

In [32]:
tx = X_train[Y_train[Y_train !=0]]

In [33]:
tx.shape

(157557, 512)

In [20]:
Y_train[t,:]

TypeError: '([2, 4, 7, 10, 11, 15, 20, 27, 30, 31, 33, 42, 43, 50, 51, 53, 54, 57, 68, 74, 75, 77, 78, 80, 85, 86, 89, 94, 96, 110, 112, 113, 120, 121, 122, 132, 134, 146, 150, 154, 155, 169, 172, 173, 174, 183, 190, 193, 196, 200, 210, 213, 214, 217, 218, 220, 226, 228, 231, 232, 233, 253, 263, 265, 266, 270, 276, 282, 284, 287, 288, 294, 297, 298, 301, 313, 314, 324, 328, 330, 331, 334, 335, 336, 347, 353, 361, 364, 367, 370, 373, 386, 393, 404, 408, 415, 423, 432, 437, 439, 444, 447, 450, 456, 457, 462, 463, 468, 469, 473, 478, 481, 483, 488, 514, 516, 520, 530, 531, 534, 536, 539, 541, 550, 553, 554, 570, 581, 582, 585, 589, 596, 599, 619, 620, 622, 623, 627, 629, 631, 637, 638, 645, 647, 649, 652, 654, 658, 664, 665, 667, 669, 672, 687, 690, 695, 708, 714, 722, 724, 728, 730, 736, 737, 746, 752, 755, 760, 761, 763, 771, 774, 776, 778, 784, 785, 790, 794, 813, 826, 828, 829, 830, 833, 838, 839, 840, 860, 861, 862, 873, 877, 879, 882, 889, 891, 900, 904, 905, 908, 912, 916, 921, 930

In [39]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
# from sklearn.preprocessing import StandardScaler
from sklearn.mixture import GaussianMixture
from sklearn.metrics import classification_report

In [37]:
# set(list(Y_train[:,0]))

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13}

In [95]:
# pos_preds = []
# neg_preds = []

# pos_preds_probs = []
# neg_preds_probs = []

# gmm_pos_models   = []
# gmm_neg_models   = []
# for i in range(14):
#     y_pos_train = Y_train[Y_train == i]
#     x_pos_train = X_train[Y_train[Y_train == i]]

#     y_neg_train = Y_train[Y_train != i]
#     x_neg_train = X_train[Y_train[Y_train != i]]
    
#     gmm_pos = GaussianMixture(n_components=3,init_params='kmeans',covariance_type='diagonal',tol=0.001).fit(x_pos_train)
#     y_pred_train = gmm_pos.predict(x_pos_train)

#     gmm_neg = GaussianMixture(n_components=3,init_params='kmeans',covariance_type='diagonal',tol=0.001).fit(x_neg_train)
#     y_pred_train = gmm_neg.predict(x_neg_train)

#     y_pred_test_pos = gmm_pos.predict(X_test)
#     pos_preds.append(y_pred_test_pos)
#     pos_preds_probs.append(gmm_pos.score)
    
#     y_pred_train_neg = gmm_neg.predict(X_test)
#     neg_preds.append(y_pred_test_neg)
#     neg_preds_probs.append(gmm_neg.score)
    
#     gmm_pos_models.append(gmm_pos)
#     gmm_neg_models.append(gmm_neg)

In [106]:
pos_preds = []
# neg_preds = []

pos_preds_probs = []
# neg_preds_probs = []

gmm_pos_models   = []
# gmm_neg_models   = []
for i in range(14):
    y_pos_train = Y_train[Y_train == i]
    x_pos_train = X_train[Y_train[Y_train == i]]

#     y_neg_train = Y_train[Y_train != i]
#     x_neg_train = X_train[Y_train[Y_train != i]]
    
    gmm_pos = GaussianMixture(n_components=8,init_params='random',covariance_type='diag',tol=0.001).fit(x_pos_train)
    y_pred_train = gmm_pos.predict(x_pos_train)

#     gmm_neg = GaussianMixture(n_components=3,init_params='kmeans',covariance_type='diag',tol=0.001).fit(x_neg_train)
#     y_pred_train = gmm_neg.predict(x_neg_train)

#     y_pred_test_pos = gmm_pos.predict(X_test)
#     pos_preds.append(y_pred_test_pos)
    pos_preds_probs.append(np.max(gmm_pos.predict_proba(X_test), axis=1))
    
#     y_pred_train_neg = gmm_neg.predict(X_test)
#     neg_preds.append(y_pred_test_neg)
#     neg_preds_probs.append(gmm_neg.score)
    
    gmm_pos_models.append(gmm_pos)
#     gmm_neg_models.append(gmm_neg)

    print("model {} is trained".format(i) )

model 0 is trained
model 1 is trained
model 2 is trained
model 3 is trained
model 4 is trained
model 5 is trained
model 6 is trained
model 7 is trained
model 8 is trained
model 9 is trained
model 10 is trained
model 11 is trained
model 12 is trained
model 13 is trained


In [107]:
pos_preds_probs = np.array(pos_preds_probs)
final_preds = np.argmax(pos_preds_probs, axis = 0)

In [108]:
print(set(list(final_preds)))

{3}


In [105]:
report_test = classification_report(Y_test.reshape(-1,), final_preds)

/jet/home/elvinj/.conda/envs/deepfix/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/jet/home/elvinj/.conda/envs/deepfix/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/jet/home/elvinj/.conda/envs/deepfix/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, le

In [59]:
y_pos_train = Y_train[Y_train == 0]
x_pos_train = X_train[Y_train[Y_train == 0]]

In [60]:
gmm_pos = GaussianMixture(n_components=4,init_params='random',covariance_type='diag',tol=0.001).fit(x_pos_train)

In [61]:
# X_test

In [62]:
y_pred_train = gmm_pos.predict(X_test)

In [63]:
y_pred_train

array([1, 1, 1, ..., 1, 1, 1])

In [66]:
gmm_pos.score_samples(X_test)

array([-1053640.30870183,  -823131.32652265,  -956031.26081529, ...,
        -972458.7931186 , -1009239.62472303,  -877415.45990513])

In [4]:
# normalised = pd.DataFrame(normalize(X_train, axis = 0))
# std_scale = sklearn.preprocessing.StandardScaler().fit(X_train)
# X_train = std_scale.transform(X_train)
# X_test  = std_scale.transform(X_test)
# 2. Finding covariance matrix
# X_train = pd.DataFrame(X_train)
# X_test  = pd.DataFrame(X_test)

# covariance_df = X_train.cov()
# # 3. Eigen Vectors
# # print(normalised.shape)
# u, s, v = np.linalg.svd(covariance_df)
# # 4. Principal Components
# # data_reduced = normalised @ u[:620]
# # print(u.shape)
# X_train = pd.DataFrame(X_train@u[:,:1024])
# X_test =  pd.DataFrame(X_test@u[:,:1024])

# data_reduced.head()

# data_reduced.to_csv("pc" + WAVELET + str(MAX_LEVEL) + ".csv", index=False)
# pcs = pd.read_csv("pc" + WAVELET + str(MAX_LEVEL)+ ".csv")
# outputs = pd.read_csv("outputs_main.csv", header=None)
# X = pcs.iloc[:, :].values
# Y = outputs.iloc[:, :].value
# X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state=42)

In [110]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
# from sklearn.preprocessing import StandardScaler
from sklearn.mixture import GaussianMixture
from sklearn.metrics import classification_report

In [112]:
gmm=GaussianMixture(n_components=14,init_params='kmeans',covariance_type='diag',tol=0.001).fit(X_train)
y_pred_train=gmm.predict(X_train)
y_pred_test=gmm.predict(X_test)


report_test = classification_report(Y_test, y_pred_test)
report_train = classification_report(Y_train, y_pred_train)

print('GMM Train',report_train)
print('GMM Test',report_test)

# svc = SVC(kernel='linear')
# svc.fit(X_train, Y_train)
# accuracy = svc.score(X_test, Y_test)
# print("Accuracy on the training set is: {0:.1f}%".format(accuracy_train*100))
# print("Accuracy on the testing set is: {0:.1f}%".format(accuracy*100))
# prediction = svc.predict(X_test)
# report = classification_report(Y_test, prediction)
# print('SVM',report)

GMM Train               precision    recall  f1-score   support

           0       0.12      0.11      0.11     21175
           1       0.08      0.05      0.06     15540
           2       0.11      0.08      0.09     19297
           3       0.22      0.06      0.09     38828
           4       0.01      0.05      0.02      2928
           5       0.04      0.02      0.03     19503
           6       0.12      0.13      0.12     20850
           7       0.06      0.04      0.05      9721
           8       0.06      0.11      0.08     10807
           9       0.03      0.04      0.04      6253
          10       0.05      0.10      0.06      9295
          11       0.00      0.02      0.00       638
          12       0.01      0.10      0.03      1906
          13       0.01      0.02      0.01      1991

    accuracy                           0.07    178732
   macro avg       0.07      0.07      0.06    178732
weighted avg       0.11      0.07      0.08    178732

GMM Test       

In [ ]:
# svc = SVC()
# parameters = {"C": (100, 1e3, 1e4, 1e5, 1e6, 1e7, 1e8, 1e9), "gamma": (1e-08, 1e-7, 1e-6, 1e-5),"kernel":("linear", "rbf","poly")}
# grid_search = GridSearchCV(svc, parameters, n_jobs=-1, cv=5)
# start_time = timeit.default_timer()
# grid_search.fit(X_train, Y_train)
# #     print("--- {0:.3f} seconds ---".format(timeit.default_timer() - start_time))
# print(grid_search.best_params_)
# svc_best = grid_search.best_estimator_
# accuracy = svc_best.score(X_test, Y_test)
# accuracy_train = svc_best.score(X_train, Y_train)
# print("Accuracy on the training set is: {0:.1f}%".format(accuracy_train*100))
# print("Accuracy on the testing set is: {0:.1f}%".format(accuracy*100))
# prediction = svc_best.predict(X_test)
# report = classification_report(Y_test, prediction)
# print('RESULTS FOR WAVELET: ',WAVELET)
# print('SVM',report)

# svc = SVC(C=10000,gamma=1e-08,kernel='poly')
# svc.fit(X_train, Y_train)
# accuracy = svc.score(X_test, Y_test)
# print("Accuracy on the training set is: {0:.1f}%".format(accuracy_train*100))
# print("Accuracy on the testing set is: {0:.1f}%".format(accuracy*100))
# prediction = svc.predict(X_test)
# report = classification_report(Y_test, prediction)
# print('SVM',report)

svc = SVC(kernel='linear')
svc.fit(X_train, Y_train)
accuracy = svc.score(X_test, Y_test)
print("Accuracy on the training set is: {0:.1f}%".format(accuracy_train*100))
print("Accuracy on the testing set is: {0:.1f}%".format(accuracy*100))
prediction = svc.predict(X_test)
report = classification_report(Y_test, prediction)
print('SVM',report)

svc = SVC(kernel='rbf')
svc.fit(X_train, Y_train)
accuracy = svc.score(X_test, Y_test)
print("Accuracy on the training set is: {0:.1f}%".format(accuracy_train*100))
print("Accuracy on the testing set is: {0:.1f}%".format(accuracy*100))
prediction = svc.predict(X_test)
report = classification_report(Y_test, prediction)
print('SVM',report)


# svc = SVC(C=10000,gamma=1e-08,kernel='linear')
# svc.fit(X_train, Y_train)
# accuracy = svc.score(X_test, Y_test)
# print("Accuracy on the training set is: {0:.1f}%".format(accuracy_train*100))
# print("Accuracy on the testing set is: {0:.1f}%".format(accuracy*100))
# prediction = svc.predict(X_test)
# report = classification_report(Y_test, prediction)
# print('SVM',report)

# svc = SVC(C=10000,gamma=1e-08,kernel='rbf')
# svc.fit(X_train, Y_train)
# accuracy = svc.score(X_test, Y_test)
# print("Accuracy on the training set is: {0:.1f}%".format(accuracy_train*100))
# print("Accuracy on the testing set is: {0:.1f}%".format(accuracy*100))
# prediction = svc.predict(X_test)
# report = classification_report(Y_test, prediction)
# print('SVM',report)